In [1]:
import pandas as pd
import os
import glob

In [2]:
# Working directory
project_dir = '/zfs/projects/darc/wolee_edehaan_suzienoh-exploratory-ml'
os.chdir(project_dir)

In [3]:
def get_prediction_file_paths(directory):
    # Use glob to get all csv files in the directory
    csv_files = glob.glob(os.path.join(directory, '*prediction*.csv'))
    return csv_files

In [4]:
def postprocess_predictions(prediction_df, prediction_col='pred', period='month'):
    
    # Target name based on period
    if period == 'quarter':
        target = 'retq'
    elif period == 'month':
        target = 'ret'
    else:
        raise ValueError("period must be 'quarter' or 'month'")
        
    prediction_df['prob']=prediction_df[prediction_col]
    prediction_df.sort_values('prob', inplace=True)
    prediction_df['rank'] = prediction_df.groupby(['date'])['prob'].transform(lambda x: pd.qcut(x.values, 10, labels=False, duplicates='drop'))
    prediction_df['port_size'] = prediction_df.groupby(['date','rank'])['mve_m'].transform('sum')
    prediction_df['port_ret'] = prediction_df[target] * prediction_df['mve_m']/prediction_df['port_size'] 

    year_vret = prediction_df.groupby(['date','rank'])['port_ret'].sum()
    year_vret = year_vret.reset_index()
    
    return year_vret

In [35]:
def maximum_return(prediction_df, prediction_col='ret', period='month'):
    
    # Target name based on period
    if period == 'quarter':
        target = 'retq'
    elif period == 'month':
        target = 'ret'
    else:
        raise ValueError("period must be 'quarter' or 'month'")
        
    prediction_df['prob']=prediction_df[prediction_col]
    prediction_df.sort_values('prob', inplace=True)
    prediction_df['rank'] = prediction_df.groupby(['date'])['prob'].transform(lambda x: pd.qcut(x.values, 10, labels=False, duplicates='drop'))
    prediction_df['port_size'] = prediction_df.groupby(['date','rank'])['mve_m'].transform('sum')
    prediction_df['port_ret'] = prediction_df[target] * prediction_df['mve_m']/prediction_df['port_size'] 

    year_vret = prediction_df.groupby(['date','rank'])['port_ret'].sum()
    year_vret = year_vret.reset_index()
    
    return year_vret

In [36]:
def create_result(prediction_parent_path, result_file_name=None, period='month', max_return=False):
    # Get the prediction data paths
    prediction_data_paths = get_prediction_file_paths(prediction_parent_path)
    
    # Postprocess the prediction and append all the results together
    results = pd.DataFrame()
    for df_path in prediction_data_paths:
        df = pd.read_csv(df_path)
        if max_return:
            if period=='month':
                year_vret = maximum_return(df, prediction_col='ret', period='month')
            elif period=='quarter':
                year_vret = maximum_return(df, prediction_col='retq', period='quarter')
            else:
                print('Please input period as either month or quarter')
        else:
            year_vret = postprocess_predictions(df, period=period)
        results = pd.concat([results, year_vret]).reset_index(drop=True)
    
    # Sort the results
    sorted_results = results.sort_values(by=['date', 'rank'],  ascending=[True, True]).reset_index(drop=True)
    
    # Save the sorted results to the same parent directory if file name is given
    if result_file_name:
        sorted_results.to_csv(f'{prediction_parent_path}/{result_file_name}', index=False)
        
    return sorted_results

In [37]:
def compute_returns(path):
    
    # Load data
    df = pd.read_csv(path)

    # Get an average annual  return for each decile
    df_avg = pd.DataFrame(df.groupby('rank')['port_ret'].mean()).reset_index().rename(columns={'port_ret': 'Average of port_ret'})

    # Get the column average for return rates
    df_avg.loc['Return rate']= df_avg.iloc[9] - df_avg.iloc[0]
    df_avg.at['Return rate', 'rank'] = ''
    df_avg = df_avg.fillna('')

    return df_avg

In [38]:
def compute_returns_df(df):
    
    # Get an average annual  return for each decile
    df_avg = pd.DataFrame(df.groupby('rank')['port_ret'].mean()).reset_index().rename(columns={'port_ret': 'Average of port_ret'})

    # Get the column average for return rates
    df_avg.loc['Return rate']= df_avg.iloc[9] - df_avg.iloc[0]
    df_avg.at['Return rate', 'rank'] = ''
    df_avg = df_avg.fillna('')

    return df_avg

In [50]:
period = 'month'
prediction_parent_path = f'kevin/output/prediction/{period}ly_old_vars'
result_file_name = 'result.csv'

In [52]:
results = create_result(prediction_parent_path, result_file_name, period, max_return=False)

In [53]:
results

,date,rank,port_ret
0,1985-01,0,0.166679
1,1985-01,1,0.113476
2,1985-01,2,0.106049
3,1985-01,3,0.084016
4,1985-01,4,0.079557
...,...,...,...
4315,2020-12,5,0.034821
4316,2020-12,6,0.043869
4317,2020-12,7,0.036535
4318,2020-12,8,0.057404


In [54]:
compute_returns_df(results)

/tmp/ipykernel_2678084/4045607825.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_avg.at['Return rate', 'rank'] = ''


,rank,Average of port_ret
0,0.0,0.003681
1,1.0,0.007112
2,2.0,0.009797
3,3.0,0.009424
4,4.0,0.011221
5,5.0,0.011056
6,6.0,0.011664
7,7.0,0.010712
8,8.0,0.012864
9,9.0,0.017309


In [24]:
compute_returns('/zfs/projects/darc/wolee_edehaan_suzienoh-exploratory-ml/kevin/output/prediction/monthly_new_vars/result.csv')

/tmp/ipykernel_2678084/2335035349.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_avg.at['Return rate', 'rank'] = ''


,rank,Average of port_ret
0,0.0,0.005073
1,1.0,0.008334
2,2.0,0.010095
3,3.0,0.010380
4,4.0,0.011332
5,5.0,0.013042
6,6.0,0.011259
7,7.0,0.011464
8,8.0,0.012775
9,9.0,0.014816


In [25]:
compute_returns('/zfs/projects/darc/wolee_edehaan_suzienoh-exploratory-ml/kevin/output/prediction/quarterly_new_vars/result.csv')

/tmp/ipykernel_2678084/2335035349.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_avg.at['Return rate', 'rank'] = ''


,rank,Average of port_ret
0,0.0,0.028847
1,1.0,0.026470
2,2.0,0.031129
3,3.0,0.030091
4,4.0,0.031548
5,5.0,0.035604
6,6.0,0.034563
7,7.0,0.032160
8,8.0,0.032736
9,9.0,0.031680


In [26]:
compute_returns('/zfs/projects/darc/wolee_edehaan_suzienoh-exploratory-ml/kevin/output/prediction/monthly_old_vars/result.csv')

/tmp/ipykernel_2678084/2335035349.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_avg.at['Return rate', 'rank'] = ''


,rank,Average of port_ret
0,0.0,0.003681
1,1.0,0.007112
2,2.0,0.009797
3,3.0,0.009424
4,4.0,0.011221
5,5.0,0.011056
6,6.0,0.011664
7,7.0,0.010712
8,8.0,0.012864
9,9.0,0.017309
